# Bonus Unit 1: Let's train Huggy the Dog to fetch a stick

### The environment 🎮

- Huggy the Dog, an environment created by [Thomas Simonini](https://twitter.com/ThomasSimonini) based on [Puppo The Corgi](https://blog.unity.com/technology/puppo-the-corgi-cuteness-overload-with-the-unity-ml-agents-toolkit)

### The library used 📚

- [MLAgents](https://github.com/Unity-Technologies/ml-agents)

- `Session options > Accelerator > GPU`

## Clone the repository and install the dependencies 🔽

- We need to clone the repository, that contains ML-Agents.

In [1]:
%%capture
# Clone the repository (can take 3min)
!git clone --depth 1 https://github.com/Unity-Technologies/ml-agents

Ignore python version, it had problems with 3.10.13

In [2]:
%%capture
# Go inside the repository and install the package (can take 3min)
%cd ml-agents
!pip3 install --ignore-requires-python -e ./ml-agents-envs
!pip3 install --ignore-requires-python -e ./ml-agents

## Download and move the environment zip file in `./trained-envs-executables/linux/`

- Our environment executable is in a zip file.
- We need to download it and place it to `./trained-envs-executables/linux/`

In [3]:
!mkdir ./trained-envs-executables
!mkdir ./trained-envs-executables/linux

We downloaded the file Huggy.zip from https://github.com/huggingface/Huggy using `wget`

In [4]:
!wget "https://github.com/huggingface/Huggy/raw/main/Huggy.zip" -O ./trained-envs-executables/linux/Huggy.zip

--2026-01-15 15:29:28--  https://github.com/huggingface/Huggy/raw/main/Huggy.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/huggingface/Huggy/main/Huggy.zip [following]
--2026-01-15 15:29:28--  https://media.githubusercontent.com/media/huggingface/Huggy/main/Huggy.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39214997 (37M) [application/zip]
Saving to: './trained-envs-executables/linux/Huggy.zip'

./trained-envs-exec 100%[===================>]  37.40M  25.1MB/s    in 1.5s    

2026-01-15 15:29:30 (25.1 MB/s) - './trained-envs-executables/linux/Huggy.zip' saved [39214

In [5]:
%%capture
!unzip -d ./trained-envs-executables/linux/ ./trained-envs-executables/linux/Huggy.zip

Make sure your file is accessible

In [6]:
!chmod -R 755 ./trained-envs-executables/linux/Huggy



### The State Space: what Huggy "perceives."

Huggy doesn't "see" his environment. Instead, we provide him information about the environment:

- The target (stick) position
- The relative position between himself and the target
- The orientation of his legs.

Given all this information, Huggy can decide which action to take next to fulfill his goal.

### The Action Space: what moves Huggy can do
Joint motors drive huggy legs. It means that to get the target, Huggy needs to learn to rotate the joint motors of each of his legs correctly so he can move.

### The Reward Function

The reward function is designed so that Huggy will fulfill his goal : fetch the stick.

Here, our goal is that Huggy goes towards the stick but without spinning too much. Hence, our reward function must translate this goal.

Our reward function:

- *Orientation bonus*: we reward him for getting close to the target.
- *Time penalty*: a fixed-time penalty given at every action to force him to get to the stick as fast as possible.
- *Rotation penalty*: we penalize Huggy if he spins too much and turns too quickly.
- *Getting to the target reward*: we reward Huggy for reaching the target.

## Create the Huggy config file

In [7]:
import yaml

In [8]:
data={
    "behaviors": {
        "Huggy": {
            "trainer_type": "ppo",
            "hyperparameters": {
                "batch_size": 2048,
                "buffer_size": 20480,
                "learning_rate": 0.0003,
                "beta": 0.005,
                "epsilon": 0.2,
                "lambd": 0.95,
                "num_epoch": 3,
                "learning_rate_schedule": "linear",
            },
            "network_settings": {
                "normalize": True,
                "hidden_units": 512,
                "num_layers": 3,
                "vis_encode_type": "simple",
            },
            "reward_signals": {"extrinsic": {"gamma": 0.995, "strength": 1.0}},
            "checkpoint_interval": 200000,
            "keep_checkpoints": 15,
            "max_steps": 2e6,
            "time_horizon": 1000,
            "summary_freq": 50000,
        }
    }
}
with open('/kaggle/working/ml-agents/config/ppo/Huggy.yaml', 'w') as f:
    yaml.dump(data, f, default_flow_style=False)

- In the case you want to modify the hyperparameters, you may edit them here

- For instance if you want to save more models during the training (for now, we save every 200,000 training timesteps). You need to modify:
  - `checkpoint_interval`: The number of training timesteps collected between each checkpoint.
  - `keep_checkpoints`: The maximum number of model checkpoints to keep.

## Train our agent

To train our agent, we just need to launch mlagents-learn and select the executable containing the environment.

With ML Agents, we run a training script. We define four parameters:

1. `mlagents-learn <config>`: the path where the hyperparameter config file is.
2. `--env`: where the environment executable is.
3. `--run-id`: the name you want to give to your training run id.
4. `--no-graphics`: to not launch the visualization during the training.

Train the model and use the `--resume` flag to continue training in case of interruption.

> It will fail first time when you use `--resume`, try running the block again to bypass the error.



The training will take 30 to 45min depending on your machine (don't forget to set up a GPU), go take a ☕️you deserve it 🤗.

In [9]:
!mlagents-learn ./config/ppo/Huggy.yaml --env=./trained-envs-executables/linux/Huggy/Huggy --run-id="Huggy2" --no-graphics

2026-01-15 15:29:51.923491: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-01-15 15:29:51.923625: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-01-15 15:29:52.066534: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

            ┐  ╖
        ╓╖╬│╡  ││╬╖╖
    ╓╖╬│││││┘  ╬│││││╬╖
 ╖╬│││││╬╜        ╙╬│││││╖╖                               ╗╗╗
 ╬╬╬╬╖││╦╖        ╖╬││╗╣╣╣╬      ╟╣╣╬    ╟╣╣╣             ╜╜╜  ╟╣╣
 ╬╬╬╬╬╬╬╬╖│╬╖╖╓╬╪│╓╣╣╣╣╣╣╣╬      ╟╣╣╬    ╟╣╣╣ ╒╣╣╖╗╣╣╣╗   ╣╣╣ ╣╣╣╣╣╣ ╟╣╣╖   ╣╣╣
 ╬╬╬╬┐  ╙╬╬╬╬│╓╣╣╣╝╜  ╫╣╣╣╬      ╟╣╣╬    ╟╣╣╣ ╟╣╣╣╙ ╙╣╣╣  ╣╣╣ ╙╟╣╣╜╙  ╫╣╣  ╟╣╣
 ╬╬╬╬┐     ╙╬╬

## Push the agent to the 🤗 Hub

- Now that we trained our agent, we’re ready to push it to the Hub to be able to play with Huggy on your browser🔥.

To be able to share your model with the community there are three more steps to follow:

1️⃣ (If it's not already done) create an account to HF ➡ https://huggingface.co/join

2️⃣ Sign in and then, you need to store your authentication token from the Hugging Face website.
- Create a new token (https://huggingface.co/settings/tokens) with write role

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="Create HF Token">

- Copy the token
- Run the cell below and paste the token

In [11]:
from huggingface_hub import notebook_login
notebook_login()

In [14]:
!git config --global credential.helper store

In [15]:
# from huggingface_hub import login

# # Replace 'your_new_write_token_here' with your actual token
# login(token='', add_to_git_credential=True)

import os
from kaggle_secrets import UserSecretsClient
from huggingface_hub import HfApi, login

# # 1. Manually delete the local token file if it exists
# token_path = os.path.expanduser("~/.cache/huggingface/token")
# if os.path.exists(token_path):
#     os.remove(token_path)
#     print("🗑️ Old cached token deleted.")

# Fetch the secret
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")


# Force login
login(token=hf_token, add_to_git_credential=True)

# 2. Verify identity and permissions
api = HfApi()
user_info = api.whoami()
print(f"✅ Successfully logged in as: {user_info['name']}")
print(f"✅ Token has Write access: {user_info['auth']['type'] == 'WRITE'}")

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful
✅ Successfully logged in as: Chiz
✅ Token has Write access: False


And we define 4 parameters:

1. `--run-id`: the name of the training run id.
2. `--local-dir`: where the agent was saved, it’s results/<run_id name>, so in my case results/First Training.
3. `--repo-id`: the name of the Hugging Face repo you want to create or update. It’s always <your huggingface username>/<the repo name>
If the repo does not exist it will be created automatically
4. `--commit-message`: since HF repos are git repository you need to define a commit message.

Use your own --repo-id

In [17]:
!mlagents-push-to-hf --run-id="HuggyTraining" --local-dir="/kaggle/working/ml-agents/results/Huggy2" --repo-id="Chiz/ppo-Huggy" --commit-message="Huggy"

[INFO] This function will create a model card and upload your HuggyTraining into HuggingFace Hub. This is a work in progress: If you encounter a bug, please send open an issue
[INFO] Pushing repo HuggyTraining to the Hugging Face Hub
Huggy-1399930.onnx:   0%|                           | 0.00/2.27M [00:00<?, ?B/s]
Huggy-1199965.pt:   0%|                             | 0.00/13.5M [00:00<?, ?B/s]

Huggy-1199965.onnx:   0%|                           | 0.00/2.27M [00:00<?, ?B/s]


Upload 25 LFS files:   0%|                               | 0/25 [00:00<?, ?it/s]



Huggy-1399930.pt:   0%|                             | 0.00/13.5M [00:00<?, ?B/s]




Huggy-1599947.onnx:   0%|                           | 0.00/2.27M [00:00<?, ?B/s]
Huggy-1199965.pt:  25%|████▉               | 3.36M/13.5M [00:00<00:00, 33.6MB/s]



Huggy-1399930.pt:  21%|████▏               | 2.87M/13.5M [00:00<00:00, 19.3MB/s]



Huggy-1399930.pt:  36%|███████▏            | 4.82M/13.5M [00:00<00:01, 7.44MB/s]
Huggy-1199965.pt:  50

Else, if everything worked you should have this at the end of the process(but with a different url 😆) :



```
Your model is pushed to the hub. You can view your model here: https://huggingface.co/ThomasSimonini/ppo-Huggy
```

It’s the link to your model repository. The repository contains a model card that explains how to use the model, your Tensorboard logs and your config file. What’s awesome is that it’s a git repository, which means you can have different commits, update your repository with a new push, open Pull Requests, etc.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/modelcard.png" alt="ml learn function" width="100%">

But now comes the best: being able to play with Huggy online 👀.

## Play with your Huggy 🐕

This step is the simplest:

- Open the game Huggy in your browser: https://huggingface.co/spaces/ThomasSimonini/Huggy

- Click on Play with my Huggy model

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/load-huggy.jpg" alt="load-huggy" width="100%">

1. In step 1, choose your model repository which is the model id (in my case ThomasSimonini/ppo-Huggy).

2. In step 2, choose what model you want to replay:
  - I have multiple ones, since we saved a model every 500000 timesteps.
  - But since I want the more recent, I choose `Huggy.onnx`

👉 What’s nice is to try with different models steps to see the improvement of the agent.

Congrats on finishing this bonus unit!

You can now sit and enjoy playing with your Huggy 🐶. And don't forget to spread the love by sharing Huggy with your friends 🤗. And if you share about it on social media, please tag us @huggingface and me @simoninithomas

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy-cover.jpeg" alt="Huggy cover" width="100%">


## Keep Learning, Stay  awesome 🤗